In [1]:

import mujoco
import time
import numpy as np
import mediapy as media
from IPython.display import clear_output
clear_output()


In [2]:
#Set Model up 

xml ="""
<mujoco model="Particle">
<!-- Degree of Freedom: 3000
     Actuators: 0
-->
  <include file="scene.xml"/>

  <option solver="CG" tolerance="1e-6" timestep=".01"/>

  <size memory="1G"/>

  <visual>
    <map stiffness="100"/>
  </visual>

  <default>
    <default class="wall">
      <geom type="plane" size=".5 .5 .05"/>
    </default>
  </default>

  <worldbody>
    <geom name="+x" class="wall" zaxis="1 0 0"  pos="-.5 0 -.25"/>
    <geom name="-x" class="wall" zaxis="-1 0 0" pos=".5 0 -.25"/>
    <geom name="+y" class="wall" zaxis="0 1 0"  pos="0 -.5 -.25"/>
    <geom name="-y" class="wall" zaxis="0 -1 0" pos="0 .5 -.25"/>
    <replicate count="10" offset=".07 0 0">
      <replicate count="10" offset="0 .07 0">
        <replicate count="10" offset="0 0 .07">
          <body pos="-.315 -.315 1">
            <joint type="slide" axis="1 0 0"/>
            <joint type="slide" axis="0 1 0"/>
            <joint type="slide" axis="0 0 1"/>
            <geom size=".025" rgba=".8 .2 .1 1" condim="1"/>
          </body>
        </replicate>
      </replicate>
    </replicate>
  </worldbody>
</mujoco>

"""


## Examples and Algorithms
### Example 1: Ball Pit

The first example aims to replicate a box of free bodied spheres. Originally the example includes a mocap body that is capable of being controlled by the users mouse. Unfortunately I was unsuccessful in recreating the mocap body. Disregarding the lack of mocap body, the spheres/balls use a joint type called "slide". The slide joint associates the balls with a position and a sliding direction. Initially in the simulation the sliding direction is only affected by gravity but as the balls hit the ground they end up creating a varity of directions (usually upward) due to different force interactions between the balls. For each ball the force equation at play is: $$F_{ball} = m_{ball} * a_{ball}$$ 
Where m is the mass of specfic ball and a is total acceleration of the ball. In this case mass has no real affect so we can just portray the force equation as $F=a$. Acceleration of the ball can be calculated by: $$a = \sqrt{a_x^2 + a_y^2 + a_z^2}$$
Initially, as stated, only gravity is at play meaning that the accleration of the ball (in the y direction) is only determined by the accleration produced by gravity ($9.81\frac{m}{s}$). As the balls hit the ground that negative accelration is turned into positive accleration. Although due to friction some energy will be lost. Not only does the ground exert a positive force but as the balls collide with each other the total force/acceleration will change. This can be summed up as:$$a_{ball} = -9.81 + j_{rebound} + n_1 + ... + n_m$$
The total accleration of a ball will be given by the summation of the gravitational force, force from ball bouncing from the floor ($j_rebound$), and any other interaction from other balls ($n_1 + ... n_m$). Where m is the total number of balls interacting with the specified ball. Do note each component/acceleration has its respective x, y and z components. 

(Author Jose Carmona)

In [38]:
#Set and visualize model (xml)
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

In [ ]:

with mujoco.viewer.launch_passive(model, data) as viewer:
  start = time.time()
  while viewer.is_running():
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

In [39]:
duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [40]:
xml_cradle = """<mujoco model="Newton's Cradle">
  <option timestep="1e-4"/>

  <visual>
    <global realtime="0.2"/>
  </visual>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="64" height="64"/>
    <texture name="texplane" type="2d" builtin="checker" rgb1=".2 .3 .4" rgb2=".1 0.15 0.2" width="512" height="512" mark="cross" markrgb=".8 .8 .8"/>
    <material name="matplane" reflectance="0.5" texture="texplane" texrepeat="3 3" texuniform="true"/>
  </asset>

  <default>
    <geom friction=".5" solref="-1e8 -0" solimp="0 .01 1e-3"/>
    <tendon limited="true" range="0 0.146" width="0.001" solreflimit="1e-3 1" rgba="1 0 .7 1"/>
    <default class="steel ball">
      <geom size=".02" rgba=".7 .7 0 1" density="8000" friction=".4" solref="-1e8 -0" solimp="0 .01 1e-3"/>
    </default>
  </default>

  <worldbody>
    <light pos=".5 0 1" dir="-1 0 -2"/>
    <light pos="-.5 0 1" dir="1 0 -2"/>
    <geom type="plane" size="4 4 .1" material="matplane"/>

    <replicate count="5" offset="-.04 0 0">
      <site name="a" pos="0 -.04 .2"/>
      <site name="c" pos="0 .04 .2"/>
      <body name="equilibrium" pos="0 0 .04">
        <freejoint/>
        <geom class="steel ball"/>
        <site name="b" pos="0 -.008 .018"/>
        <site name="d" pos="0 .008 .018"/>
      </body>
    </replicate>

    <replicate count="2" offset=".04 0 0">
      <site name="e" pos=".04 -.04 .2"/>
      <site name="g" pos=".04 .04 .2"/>
      <body name="perturbed" pos=".12 0 .061" euler="0 -34 0">
        <freejoint/>
        <geom class="steel ball"/>
        <site name="f" pos="0 -.008 .018"/>
        <site name="h" pos="0 .008 .018"/>
      </body>
    </replicate>
  </worldbody>

  <tendon>
    <spatial>
      <site site="a"/>
      <site site="b"/>
    </spatial>
    <spatial>
      <site site="c"/>
      <site site="d"/>
    </spatial>
    <spatial>
      <site site="e"/>
      <site site="f"/>
    </spatial>
    <spatial>
      <site site="g"/>
      <site site="h"/>
    </spatial>
  </tendon>

</mujoco>
"""

### Example 2: Newton's Cradle

The next example looks at simulating newton's cradle. This example implements a lot of the same components from the first example. Specfically the way in which the ball behaves where ultimately the movement of the ball will be determined by the equation seen below:$$a_{ball} = -9.81 + j_{strings} + n_1 + n_2$$
As one can see the equation is very similar to the one in example 1. Some key differences is that the balls are being held up by two strings/tendons ($j_strings$) therefore the balls are constained to a certain degree of movement specfically the balls are limited to how much they can move forward and backward (if view is parallel to how the tendons are set up). This in turn causes the acceleration that is applied to the balls to be composed of only x and z components (assuming tendons stop all movement in the y direction). Another key difference is that for a single ball there will be at most two different balls hitting against the specified ball (n_1 + n_2). Similar to example 1 gravity will always be in play (unless user turns off gravity).

(Author Jose Carmona)

In [47]:
#Set and visualize model (xml)
model = mujoco.MjModel.from_xml_string(xml_cradle)
data = mujoco.MjData(model)

In [7]:
with mujoco.viewer.launch_passive(model, data) as viewer:
  start = time.time()
  while viewer.is_running():
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

In [48]:
duration = 10  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.


In [43]:
ben_xml = xml = """<?xml version="1.0" ?>
<mujoco model="fun_simulation">
  <!-- Compiler and simulation options -->
  <compiler angle="radian"/>
  <option timestep="0.005" gravity="0 0 -9.81"/>
  <size njmax="2000" nconmax="1000"/>
  
  <worldbody>
    <!-- Ground plane -->
    <geom name="ground" type="plane" pos="0 0 0" size="500 500 0.1" rgba="1 1 1 1"/>
    
    <!-- Seesaw structure -->
    <body name="seesaw_base" pos="0 0 0.1">
      <body name="seesaw_plank" pos="0 0 0">
        <!-- Hinge joint for seesaw rotation -->
        <joint name="seesaw_hinge" type="hinge" axis="0 1 0" pos="0 0 0"/>
        <geom name="plank_geom" type="box" size="1 0.1 0.02" rgba="0.4 0.3 0.2 1"/>
      </body>
    </body>
    
    <!-- Dynamic objects placed near the seesaw -->
    <body name="ball" pos="-0.7 0 0.35">
      <joint name="ball_free" type="free"/>
      <geom name="ball_geom" type="sphere" size="0.25" rgba="0.9 0.1 0.1 1"/>
    </body>
    <body name="box" pos=".7 0 4">
      <joint name="box_free" type="free"/>
      <geom name="box_geom" type="box" size=".5 3 .5" rgba="0.1 0.1 0.9 1"/>
    </body>
    <body name="box2" pos="-.7 0 100">
      <joint name="box_free2" type="free"/>
      <geom name="box_geom2" type="box" size=".5 3 .5" rgba="0.1 0.1 0.9 1"/>
    </body>
    
    <!-- Pendulum -->
    <body name="pendulum" pos="2 0 2">
      <!-- Hinge joint at (0,0,2) in the pendulum's local frame -->
      <joint name="pendulum_hinge" type="hinge" axis="0 1 0" pos="0 0 2"/>
      <!-- Rod geometry connecting the pivot to the mass -->
      <geom name="rod" type="capsule" fromto="0 0 2 0.2 0 -1" size="0.02" rgba="0.6 0.6 0.6 1"/>
      <!-- Pendulum mass is offset horizontally to set a nonzero initial angle -->
      <body name="pendulum_mass" pos="0.2 0 -1">
        <geom name="mass" type="sphere" size="0.3" rgba="0.1 0.9 0.1 1"/>
      </body>
    </body>
    
    
    <replicate count="10" offset=".3 0 0">
      <replicate count="10" offset="0 .3 0">
        <replicate count="7" offset="0 0 .3">
          <body pos="-2 -1.35 125">
            <joint name="bf" type="free"/>
            <geom size=".15" rgba=".9 .1 .9 1" condim="1"/>
          </body>
        </replicate>
      </replicate>
    </replicate>
    
    
    <!-- Ramp: A sloped plane -->
    <body name="ramp" pos="-2 0 0.5">
      <geom name="ramp_geom" type="plane" size="500 500 0.01" quat="0 0.382683 0 0.92388" rgba="0.9 0.9 0.1 1"/>
    </body>
    
    <!-- Top-level ramp ball placed to appear on the ramp's high end -->
    <body name="ramp_ball" pos="-2 0 1">
      <joint name="ramp_ball_free" type="free"/>
      <geom name="ramp_ball_geom" type="sphere" size="0.3" rgba="0.1 0.9 0.9 1"/>
    </body>
  </worldbody>
  
  <!-- Actuators to control the dynamic joints -->
  <actuator>
    <!-- Motor applying torque to the seesaw hinge -->
    <motor joint="seesaw_hinge" ctrlrange="-1 1" ctrllimited="true" gear="50"/>
    <!-- Motor to swing the pendulum -->
    <motor joint="pendulum_hinge" ctrlrange="-1 1" ctrllimited="true" gear="10"/>
  </actuator>
</mujoco>"""

In [44]:
#Set and visualize model (xml)
model = mujoco.MjModel.from_xml_string(ben_xml)
data = mujoco.MjData(model)

In [70]:
with mujoco.viewer.launch_passive(model, data) as viewer:
  start = time.time()
  while viewer.is_running():
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)